In [12]:
# First clone and set up: https://github.com/dwalton76/rubiks-cube-NxNxN-solver.
# Then, put this notebook in the root directory of that repo.

# General libraries
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import json
from typing import Iterable
import subprocess

In [13]:
DATA_DIRECTORY = '../../santa-2023/'

In [14]:
# Rubik's Cube solver library from GitHub

from rubikscubennnsolver import RubiksCube, SolveError
import rubikscubennnsolver

# IMPORTANT CHANGES TO REPOSITORY:

# 1.
# This custom method was added to RubiksCube in file rubiks-cube-NxNxN-solver/rubikscubennnsolver/__init__.py 
"""
def get_solution(self):
    solution_minus_comments = []
    for step in self.solution:
        if not step.startswith("COMMENT"):
            solution_minus_comments.append(step)
    return solution_minus_comments
""";

# 2.
# I edited RubiksCube222 (line 113), so it would "return" instead of sys.exit(0) (error in jupyter)
# Also removed the unnecessary print right above that line

In [15]:
def get_inverse_perm(perm : list) -> list:
    inv_perm = [0] * len(perm)
    
    for i in range(len(perm)):
        inv_perm[perm[i]] = i
        
    return inv_perm

In [16]:
# Load puzzle_info into a dictionary to easily access the moves

allowed_moves = {}

puzzle_info = pd.read_csv(DATA_DIRECTORY + 'puzzle_info.csv')
for _, row in puzzle_info.iterrows():
    allowed_moves[row['puzzle_type']] = json.loads(row['allowed_moves'].replace("'", '"'))

    # Add inverse moves
    d = allowed_moves[row['puzzle_type']]
    for k, move in tuple(d.items()):
        d['-' + k] = get_inverse_perm(move)

In [17]:
def convert_to_rcube(s: str) -> str:
    """
    Convert the string to 
    1. Change the colors
    2. The faces are written in the order: 'BCDEAF'
    3. Viewing the "flattened" cube, the colors for each face are written bottom to top
    
    To figure this out, look at Cube.h:
    (1.) and (2.)
        From matching 'q' in Move() and 'step' in Rotate?()
        More specifically passing depth => index can be 0 and then look at
            a line like: if (index == 0)  faces[3].RotatefaceCW(-step);
    (3.)
        From first figuring out the X, Y, Z plane location via ideas from above.
        Then, match the index arg in Rotate?() to see which direction row/col increases.
            For example: 
                p0 = faces[0].GetPos(0, index);
                This means face 0's column increases left to right.
        In the end, all rows increase bottom to top and columns left to right.

    
    
    Invariant is that faces are written as concatenating the rows.
    
    EDGE CASE:
        The faces in rcube are written with respect to that face so top face (4)
        is flipped upside down when flattened in 2d!
    
    Kaggle:
        Flattened Coloring:
         A
        EBCD
         F
         
        String has faces in order: 'ABCDEF
     
    Rcube:
        Flattened Coloring:
         4
        3012
         5
         
        String has faces in order '012345'
    
    Side note, it took forever to figure out (3.) was true. 2.5 hrs of debugging.
    The other way to get the rcube is to use RotateX, RotateY, RotateZ + sample_submission.
    This takes the same time to code in theory, but it's safer since their code
    first calls .Scramble() which only uses Rotate? before calling solve().
    Thus, it's guaranteed to work.
    This method requires more assumptions, like (3.).
    """
    
    convert_color = {'A' : '4',
                     'B' : '0',
                     'C' : '1',
                     'D' : '2',
                     'E' : '3',
                     'F' : '5'}

    s = list(s)
    m = len(s) // 6
    n = int(m**0.5)

    # 1
    for i in range(len(s)):
        s[i] = convert_color[s[i]]

    # 2
    s = [s[m*i:m*(i+1)] for i in range(6)]
    perm = [1, 2, 3, 4, 0, 5]
    s = [s[perm[i]] for i in range(6)]
    
    # 3
    for i in range(6):
        new_order = []
        for row in range(n-1, -1, -1):
            new_order.extend(s[i][row*n:(row+1)*n])
        s[i] = new_order
            
    s = [s[i//m][i%m] for i in range(6*m)]

    return ''.join(s)

In [18]:
def convert_to_kociemba(s: str) -> str:
    """
    Convert the string to kociemba notation
    1. Change the colors
    2. The faces are written in the order: 'URFDLB'
    
    Kaggle Coloring:
    (https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023?scriptVersionId=155960527&cellId=14)
             +--------+                               +--------+
             | 0    1 |                               | A    A |
             |   d1   |                               |   d1   |
             | 2    3 |                               | A    A |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
    | 16  17 | 4    5 | 8   9  | 12  13 |    | E    E | B    B | C    C | D    D |
    |   r1   |   f0   |   r0   |   f1   |    |   r1   |   f0   |   r0   |   f1   |
    | 18  19 | 6    7 | 10  11 | 14  15 |    | E    E | B    B | C    C | D    D |
    +--------+--------+--------+--------+    +--------+--------+--------+--------+
             | 20  21 |                               | F    F |
             |   d0   |                               |   d0   |
             | 22  23 |                               | F    F |
             +--------+                               +--------+
    
    Kociemba Coloring:
    (https://github.com/muodov/kociemba?tab=readme-ov-file#cube-string-notation)
                 |************|
                 |*U1**U2**U3*|
                 |************|
                 |*U4**U5**U6*|
                 |************|
                 |*U7**U8**U9*|
                 |************|
     ************|************|************|************
     *L1**L2**L3*|*F1**F2**F3*|*R1**R2**R3*|*B1**B2**B3*
     ************|************|************|************
     *L4**L5**L6*|*F4**F5**F6*|*R4**R5**R6*|*B4**B5**B6*
     ************|************|************|************
     *L7**L8**L9*|*F7**F8**F9*|*R7**R8**R9*|*B7**B8**B9*
     ************|************|************|************
                 |************|
                 |*D1**D2**D3*|
                 |************|
                 |*D4**D5**D6*|
                 |************|
                 |*D7**D8**D9*|
                 |************|
    """
    
    convert_color = {'A' : 'U', 
                     'B' : 'F', 
                     'C' : 'R',
                     'D' : 'B',
                     'E' : 'L',
                     'F' : 'D'}
    
    s = list(s)
    m = len(s) // 6
    
    # 1
    for i in range(len(s)):
        s[i] = convert_color[s[i]]
    
    # 2
    s = [s[m*i:m*(i+1)] for i in range(6)]
    perm = [0, 2, 1, 5, 4, 3]
    s = [s[perm[i]] for i in range(6)]
    s = [s[i//m][i%m] for i in range(6*m)]
    
    return ''.join(s)

assert(convert_to_kociemba('AAAABBBBCCCCDDDDEEEEFFFF') == 'UUUURRRRFFFFDDDDLLLLBBBB')

In [19]:
def find_n(s : str) -> int:
    """
    Returns n where n is the size of the cube given its string representation
    """
    return int((len(s) // 6) ** 0.5) # No precision errors since float result rounded from exact result

In [20]:
def get_cube(kociemba_notation: str) -> RubiksCube:
    """
    Gets the correct cube object from kociemba notation
    """
    
    size = find_n(kociemba_notation)
    KOCIEMBA_FACE_ORDER = 'URFDLB'
    
    if size == 2:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube222 import RubiksCube222

        cube = RubiksCube222(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 3:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube333 import RubiksCube333

        cube = RubiksCube333(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 4:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube444 import RubiksCube444

        cube = RubiksCube444(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 5:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube555 import RubiksCube555

        cube = RubiksCube555(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 6:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube666 import RubiksCube666

        cube = RubiksCube666(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size == 7:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCube777 import RubiksCube777

        cube = RubiksCube777(kociemba_notation, KOCIEMBA_FACE_ORDER)
    elif size % 2 == 0:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCubeNNNEven import RubiksCubeNNNEven

        cube = RubiksCubeNNNEven(kociemba_notation, KOCIEMBA_FACE_ORDER)
    else:
        # rubiks cube libraries
        from rubikscubennnsolver.RubiksCubeNNNOdd import RubiksCubeNNNOdd

        cube = RubiksCubeNNNOdd(kociemba_notation, KOCIEMBA_FACE_ORDER)
    
    return cube

In [21]:
def convert_cuber_moves_to_kaggle_moves(moves : list[str], n) -> list[str]:
    """
    Converts the cuber move to kaggle moves
    
    To learn about cuber notation for moves:
        https://www.youtube.com/watch?v=37y4f8FYdFs
        One confusing case is that Rw = 2Rw (they are used interchangeably)
    
    To learn about kaggle moves:
        https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2023?scriptVersionId=155960527&cellId=14
        Note that wide moves don't exist on kaggle
    
    Major discrepancy:
        There is a kaggle move that rotates each layer clockwise with respect to 3 the
        FRD faces.
        However, cuber moves are clockwise with respect to one of the 6 faces.
        
    Examples:
        The R move corresponds to r_0
        The L corresponds to -r_{n-1}.
        The R2 move corresponds to r_0, r_0
        The Rw or 2Rw move corresponds to r_0, r_1
        The 2Lw move corresponds to -r_{n-1}, -r_{n-2}
    """
    
    convert_axis = {'R' : 'r',
                    'L' : 'r',
                    'F' : 'f',
                    'B' : 'f',
                    'D' : 'd',
                    'U' : 'd'}
    axis_changed = 'LBU'
    invert_str = ['', '-']
    
    
    kaggle_moves = []
    
    for move in moves:
        rotations = 1
        invert = 0
        
        if move[-1] == "'":
            invert = 1
        if move[-1] == '2':
            rotations = 2
        
        width = 1 # If no w, just rotate single layer
        
        if 'w' in move:
            width = 2  # Default if no number
            if move[0].isdigit():
                # Get and remove width from prefix of move
                width = ''
                for i in range(len(move)):
                    if not move[i].isdigit():
                        width = int(move[:i])
                        move = move[i:]
                        break

        original_axis = move[0]
        axis = convert_axis[original_axis]
        
        # Get kaggle moves
        for i in range(width):
            for j in range(rotations):
                if original_axis in axis_changed:
                    kaggle_moves.append(invert_str[invert^1] + axis + str(n-1-i))
                else:
                    kaggle_moves.append(invert_str[invert] + axis + str(i))
    
    return kaggle_moves

In [22]:
def add_final_rotation(puzzle_type: str,
                       initial_state: str,
                       moves: list[str]) -> None:
    """
    The solver from GitHub doesn't care about the final orientation of the cube.
    We'll need to rotate it.
    
    I think this uses the minimum rotations (should double-check).
    Should take at most 3*n kaggle moves (3 rotations).
    
    Updates moves in place.
    """

    n = find_n(initial_state)
    m = n*n

    current_state = np.array(tuple(initial_state))
    
    for move in moves:
        current_state = current_state[allowed_moves[puzzle_type][move]]

    new_moves = []
    b_at_3 = False
    
    # Get B in place
    # Rotation[i] = Move(s) to get B in place if they are at face i in kaggle order
    rotation = [('-r',), tuple(), ('-d',), ('d', 'r'), ('d',), ('r',)]  
    for i in range(6):
        if current_state[i*m] == 'B' and rotation[i]:
            b_at_3 = i == 3
            rotations = 2 if i == 3 else 1  # B is opposite where it should be
            for j in range(rotations):
                new_moves.extend(rotation[i][0] + str(k) for k in range(n))
    
    # Apply moves
    current_state = current_state.copy()
    for move in new_moves:
        current_state = current_state[allowed_moves[puzzle_type][move]]
    
    # Get 'C' in place
    if b_at_3 and current_state[4*m] == 'C':
        # We rotated on i = 3 to get B in place.
        # If we use the other valid rotation, it puts B AND C in place.
        new_moves = []
        for j in range(2):
            new_moves.extend(rotation[3][1] + str(k) for k in range(n))
    else:
        rotation = ['f', None, '', None, 'f', '-f']  # Notice how we only rotate about f axis
        for i in range(6):
            if current_state[i*m] == 'C' and rotation[i] != '':
                rotations = 2 if i == 4 else 1  # C is opposite where it should be 
                for j in range(rotations):
                    new_moves.extend(rotation[i] + str(k) for k in range(n))
    
    moves += new_moves

In [23]:
def invert_kaggle_moves(moves : list[str]) -> list[str]:
    """
    Gives a list of moves that when applied in order has the inverse effect
    of the original list of moves applied in order
    """
    
    inverse_moves = []
    
    for move in moves[::-1]:
        if move[0] == '-':
            inverse_moves.append(move[1:])
        else:
            inverse_moves.append('-' + move)
        
    return inverse_moves

In [24]:
def apply_moves(initial_state : Iterable[str], moves : list[str]) -> list[str]:
    current_state = np.array(tuple(initial_state))
    
    for move in moves:
        current_state = current_state[allowed_moves[puzzle_type][move]]
    
    return current_state.tolist()

In [25]:
def solve1_dwalton(puzzle_type: str,
           initial_state: str,
           solution_state: str) -> list[str]:
    """
    Solves the type 1 Puzzle: a "normal" cube puzzle.
    
    These are puzzles with solution state:
          AA
          AA
        EEBBCCDD
        EEBBCCDD
          FF
          FF
    
    We use the dwalton solver + translate between kaggle and cuber notations.
    
    Returns the kaggle moves that solves this puzzle.
    
    Efficient on small cubes but bad on larger ones since it uses wide moves. 
        Neither I nor the author knows how to directly fix this:
        https://github.com/dwalton76/rubiks-cube-NxNxN-solver/issues/98
    Thus, we have an extra O(n) multiplication factor from translating to kaggle moves.
    """

    initial_state_kociemba = convert_to_kociemba(initial_state)
    
    initial_cube = get_cube(initial_state_kociemba)
    
    initial_cube.solve()
    
    initial_cuber_moves = initial_cube.get_solution()
    
    n = find_n(initial_state)
    initial_kaggle_moves = convert_cuber_moves_to_kaggle_moves(initial_cuber_moves, n)
    
    add_final_rotation(puzzle_type, initial_state, initial_kaggle_moves)

    return initial_kaggle_moves

In [26]:
def solve1_rcube(puzzle_type: str,
                   initial_state: str,
                   solution_state: str) -> list[str]:
    """
    Solves the type 1 Puzzle: a "normal" cube puzzle.
    
    These are puzzles with solution state:
          AA
          AA
        EEBBCCDD
        EEBBCCDD
          FF
          FF
    
    We use the RCube solver + translate between kaggle and cuber notations.
          
    Returns the kaggle moves that solves this puzzle.
    
    This massively outperforms dwalton on large cubes because it uses single rotations.
    However, it only optimizes to solve centers and has a bad algo for edges / corners.
    """

    initial_state_rcube = convert_to_rcube(initial_state)

    result = subprocess.run(args=['./RCube.exe', initial_state_rcube], capture_output=True, text=True)

    initial_moves = result.stdout.split('.') if result.stdout else []

    return initial_moves

In [27]:
def solve1_hybrid(puzzle_type: str,
           initial_state: str,
           solution_state: str) -> list[str]:
    """
    Solves the type 1 Puzzle: a "normal" cube puzzle.
    
    These are puzzles with solution state:
          AA
          AA
        EEBBCCDD
        EEBBCCDD
          FF
          FF
    
    We can just use a hybrid RCube + dwalton solver then translate between kaggle and cuber notations.
        
    Returns the kaggle moves that solves this puzzle.
    
    MOTIVATION:
        For 3x3x3: RCube takes 500 moves, dwalton takes 30
        For 10x10x10: RCube takes 5000 moves, dwalton takes 2000
        
        The reason for this is that RCube optimized to solve centers and has
        a massively inefficient algorithm afterward for edges/corners.
        
        Dwalton is very efficient on small cubes, but due to the O(n) multiplication
        factor from translation, it is bad on larger ones.
        
    SOLUTION:
        Simply use RCube to solve the centers then let dwalton take over for the rest!
        
        I looked through the code to verify dwalton checks if the centers are solved.
        So, there isn't inefficiency in dwalton not "knowing" the centers are already solved.
    
    RESULTS:
        For 10x10x10: Hybrid takes 1800 moves, dwalton takes 1300
        For 19x19x19: Hybrid takes 8500 moves, dwalton takes 12000
        For 33x33x33: Hybrid takes 23500 moves, dwalton takes 26500
        
        Also, note that there is some (parity like OLL PLL or smth?) issue for even cubes
        of, n > 6.
            This is because now we use RubiksCubeNNNEven instead of RubiksCube{n}{n}{n}).
        So, dwalton often gets an error when running ida_search_via_graph from command line. However, pure dwalton seems better than hybrid in these cases anyway (yay!).
    """
    
    initial_state_rcube = convert_to_rcube(initial_state)
    
    result = subprocess.run(args=['./RCube_Centers_Only.exe', initial_state_rcube], capture_output=True, text=True)
    
    initial_moves_centers = result.stdout.split('.') if result.stdout else []
    
    centers_state = apply_moves(initial_state, initial_moves_centers)
    
    initial_moves_remaining = solve1_dwalton(puzzle_type, centers_state, solution_state)
    
    return initial_moves_centers + initial_moves_remaining

In [28]:
b = None
def solve2(puzzle_type: str,
           initial_state: str,
           solution_state: str) -> list[str]:
    """
    Solves the type 2 puzzle:
    
        The reason for this is that every internal (non corner/edge) square 
        can end up in 2 places (ignoring which face): the current place or its
    
    1/2 chance a rectangle is wrong
    Fixes 8 with 12 moves
    6*(n-2)^2 * 1/2 * 12/8 = 4.5 * (n-2)^2 moves
        
    Returns the kaggle moves that solves this puzzle.
    """

    m = len(initial_state) // 6

    recolored_solution_state = 'A'*m + 'B'*m + 'C'*m + 'D'*m + 'E'*m + 'F'*m  # By definition
    recolored_initial_state = ''.join(recolored_solution_state[int(x[1:])] for x in initial_state)
    
    moves = solve1_rcube(puzzle_type, recolored_initial_state, recolored_solution_state)
    
    # Fix rectangle corners
    
    # We only have to fix the first 3 faces since by symmetry, the rest are fine
    # See loop to better understand this "compressed" algo representation
    algos = (('rfr', ((1, 1, 1), (0, 1, 0))),
            ('rdr', ((1, 1, 1), (0, 1, 0))),
            ('fdf', ((0, 1, 0), (1, 1, 1))))
    
    final_state = apply_moves(initial_state, moves)
    global b
    b = final_state.copy()
    n = find_n(initial_state)
    
    for face, algo in enumerate(algos):
        direction, from_end = algo
        
        for row in range(n):
            for col in range(n):
                state_i = face*m + row*n + col
                
                if int(final_state[state_i][1:]) != state_i:
                    # Fix rectangle corners
                    for stage in range(2):
                        for move in range(3):
                            for rotation in range(2):
                                offset = row if move == 1 else col
                                layer = n-1-offset if from_end[stage][move] else offset
                                new_move = [direction[move] + str(layer)]
                                
                                final_state = apply_moves(final_state, new_move)
                                moves += new_move
                                    
    return moves

In [29]:
def solve3(puzzle_type: str,
           initial_state: str,
           solution_state: str) -> list[str]:
    """
    Solves the type 3 puzzle: Semi checkerboard solution and n is odd.
    
    These are puzzles with solution state:
              A;B;A
              B;A;B
              A;B;A
        E;F;E;B;C;B;C;D;C;D;E;D
        F;E;F;C;B;C;D;C;D;E;D;E
        E;F;E;B;C;B;C;D;C;D;E;D
              F;A;F
              A;F;A
              F;A;F
    
    ISSUE:
        One's first thought might be to solve the solution cube and the initial cube.
        Then, we can combine these two paths to get the moves from initial to solution.
        However, neither of the cubes are solvable (actually either both are or none are)!
        
    SOLUTION:
        To fix this we want to recolor the cube.
        Naively recoloring the solution state to the standard solved cube works!
        
        Proof:
        Note that this coloring means that for each original color, 
        there are 2 possible colors for those squares.
        Which color depends on parity of (row+column) of the square.
        
        From the solution to type 2 (read documentation of solve2), each
        internal square has 4 possible end positions. 
        However, since they are 90 degree rotations, the parity of (row + column) is the same.
        Then, those 4 end positions have the same original color.
        So, no matter how these 4 end up permuted after solving, the original colors are the same.
        Thus, after solving, if we revert to the original colors, we get the initial cube!
    
    Returns the kaggle moves that solves this puzzle.
    """
    
    n = find_n(initial_state)
    m = n*n

    # Recolor to reduce to type 1 puzzle
    # We could also use the given sample moves like in solve4, but this illustrates idea better 
    # (also works if we didn't have sample moves)
    
    # Unique new color based on the color and parity of (row_number + column_number)
    new_color = {}
    
    standard_solved_order = 'ABCDEF'
    # Iterate over the 6 faces and the first 2 elements of the first row
    # This goes over all types of (color, parity) pairs since type 3 is checkerboard
    for i in range(6):
        for j in range(2):
            new_color[(solution_state[i*m+j], j)] = standard_solved_order[i]

    recolored_solution_state = 'A'*m + 'B'*m + 'C'*m + 'D'*m + 'E'*m + 'F'*m  # By definition
    recolored_initial_state = []
    
    for face in range(6):
        for row in range(n):
            for col in range(n):
                i = face*m + row*n + col
                recolored_initial_state.append(new_color[(initial_state[i], (row+col)&1)])
    
    # Now we've reduced the puzzle to a type 1 puzzle
    return solve1_hybrid(puzzle_type, recolored_initial_state, recolored_solution_state)

In [30]:
def solve4(puzzle_type: str,
           initial_state: str,
           solution_state: str,
           sample_moves) -> list[str]:
    """
    Solves the type 4 puzzle:
    Returns the kaggle moves that solves this puzzle.
    """
    
    # Recolor to reduce to type 2 puzzle
    recolored_solution_state = [f'N{i}' for i in range(len(initial_state))]
    recolored_initial_state = apply_moves(recolored_solution_state, invert_kaggle_moves(sample_moves))
    
    return solve2(puzzle_type, recolored_initial_state, recolored_solution_state)

In [31]:
def shorten_redundant_moves(moves: list[str]) -> list[str]:
    """
    WLOG consider the layer corresponding to r1.
    Let M be the moves of r1 or -r1 up until we have some move with f or d.
    In other words, we consider the moves that directly rotate the layer up until a move that affects it.
    
    If the net rotations for M is:
        0: We delete all moves
        1: Replace the moves with r1
        2: Replace the moves with r1, r1
        3: Replace the moves with -r1
    
    1. These redundant moves can be a byproduct of translating from cuber moves to kaggle moves:
        To rotate some internal subsegment, the solver uses 2 wide moves.
        Also, (2Rw, R) -> (r0, r0, r1, r1, r0) -> (r1, -r0). Note there is no shorter cuber sequence.
    2. Of course, the solver is sometimes inefficient.
    
    Reduces the moves by ~5% for rubiks-cube-NxNxN... repo
    Reduces the moves by 40% for RCube
    Reduces the moves by 30% for hybrid
    
    O(3*n*|moves|)
    """
    
    visited = [0] * len(moves)
    delete = [0] * len(moves)
    replace = [None] * len(moves)
    
    for i in range(len(moves)):
        if not visited[i]:
            visited[i] = 1
            
            moves_at_layer = [i]
            move = moves[i]
            net_rotation = 1
            
            if move[0] == '-':
                net_rotation = -1
                move = move[1:]
                
            for j in range(i+1, len(moves)):
                later_move = moves[j]
                rotation = 1
                if later_move[0] == '-':
                    rotation = -1
                    later_move = later_move[1:]
                
                if later_move[0] != move[0]:
                    # We affected the layer
                    net_rotation %= 4
                    invert = ''
                    
                    if net_rotation == 3:
                        net_rotation = 1
                        invert = '-'
                    
                    # Add len(moves_at_layer) for edge case of 0 net rotation
                    for j in moves_at_layer[:len(moves_at_layer)-net_rotation]:
                        delete[j] = 1
                    for j in moves_at_layer[len(moves_at_layer)-net_rotation:]:
                        replace[j] = invert + move
                        
                    break
                else:
                    if later_move == move:
                        visited[j] = 1
                        moves_at_layer.append(j)
                        net_rotation += rotation
    
    new_moves = moves.copy()
    for i in range(len(moves)):
        if replace[i]:
            new_moves[i] = replace[i]
    
    new_moves = [new_moves[i] for i in range(len(moves)) if not delete[i]]
    
    return new_moves

In [32]:
def shorten_until_wildcard(initial_state : str,
                           solution_state : str,
                           num_wildcards : int,
                           moves : list[str]) -> list[str]:
    """
    Applies moves until at most num_wildcard colors are off
    """
    
    if num_wildcards == 0:
        return moves

    current_state = np.array(tuple(initial_state))

    for i in range(len(moves)):
        if sum(current_state != solution_state) <= num_wildcards:
            moves = moves[:i]
            break
            
        current_state = current_state[allowed_moves[puzzle_type][moves[i]]]
    
    return moves

In [33]:
def check_moves(initial_state : str,
                solution_state : str,
                num_wildcards : int,
                moves : list[str]) -> bool:
    """
    Returns if moves converts initial_state to solution_state
    """
    
    final_state = apply_moves(initial_state, moves)
    solution_state = np.array(tuple(solution_state))
    
    return sum(final_state != solution_state) <= num_wildcards

In [34]:
def print_cube(cube : list[str]) -> None:
    """
    Prints the cube in a readable format with faces
    """
    
    n = find_n(cube)
    for i in range(0, len(cube), n):
        print(cube[i:i+n])
        if i % (n*n) == (n-1)*n:
            print()

In [35]:
# Get submission and cube puzzles

submission = pd.read_csv(DATA_DIRECTORY + 'submission.csv', index_col=0)
puzzles = pd.read_csv(DATA_DIRECTORY + 'puzzles.csv')
puzzles = puzzles[puzzles['puzzle_type'].str.contains('cube')]

In [36]:
# To quantify improvement and potential improvement

skipped_has_n = []  # Skipped because has 'N'
tot_skipped_has_n = 0
cannot_solve = []  # Skipped because meeting point of standard solved cube impossible
tot_cannot_solve = 0
improve = {}
tot_improved = 0
error_while_solving = []  # Unknown errors, but almost certainly from GitHub cube solver
cube_solver_wrong = []  # Cube solver doesn't get it right...

In [ ]:
# [205]
# [240]
for i in tqdm(range(len(puzzles))):
    puzzle_type = puzzles.iloc[i]['puzzle_type']
    num_wildcards = puzzles.iloc[i]['num_wildcards']
    initial_state = puzzles.iloc[i]['initial_state'].split(';')
    solution_state = puzzles.iloc[i]['solution_state'].split(';')
    id = puzzles.iloc[i]['id']
    old_moves = submission.iloc[id]['moves'].split('.')
    
    # Find the type of puzzle and call the correct function to solve
    try:
        if 'N' in solution_state[0]:
            # Type 2
            moves = solve2(puzzle_type, initial_state, solution_state)
        elif solution_state[1] == 'B':
            n = find_n(initial_state)
            if n & 1:
                # Type 3
                moves = solve3(puzzle_type, initial_state, solution_state)
            else:
                continue
                # Type 4
                moves = solve4(puzzle_type, initial_state, solution_state, old_moves)
        else:
            # Type 1
            moves = solve1_hybrid(puzzle_type, initial_state, solution_state)
    except:
        error_while_solving.append(id)
        continue

    # Shorten moves
    moves = shorten_until_wildcard(initial_state, solution_state, num_wildcards, moves)
    moves = shorten_redundant_moves(moves)
    
    # Verify correctness
    if not check_moves(initial_state, solution_state, num_wildcards, moves):
        print('hi')
        a = apply_moves(initial_state, moves)
        cube_solver_wrong.append((id, puzzle_type, len(old_moves)))
    else:
        # Compare and replace if better
        if len(moves) < len(old_moves):
            submission.loc[id, 'moves'] = '.'.join(moves)  # Replace
            
            improvement = len(old_moves) - len(moves)
            
            # Print improvement
            print(f'Improved {puzzle_type} with id {id} by {improvement} moves')
            
            # Store improvement statistics
            if puzzle_type not in improve:
                improve[puzzle_type] = [0, 0, 0]
            improve[puzzle_type][0] += improvement
            improve[puzzle_type][1] += len(old_moves)
            improve[puzzle_type][2] += 1
            
            tot_improved += improvement

In [ ]:
print_cube(b)

In [ ]:
print_cube(a)

In [ ]:
print(f'{tot_improved=}')

In [ ]:
print(f'{tot_skipped_has_n=}')
print(f'{skipped_has_n=}')

In [ ]:
print(f'{tot_cannot_solve}')
print(f'{cannot_solve=}')

In [ ]:
print(f'{error_while_solving=}')
print(f'{cube_solver_wrong=}')

In [ ]:
redo = [a[0] for a in error_while_solving] + [a[0] for a in cube_solver_wrong]
print(f'{redo=}')

In [ ]:
for puzzle_type, v in improve.items():
    print(f'{puzzle_type} Average Improvement: {v[0]/v[2]} Score, {100*v[0]/v[1]}%')

In [ ]:
# Save submission

submission.to_csv(DATA_DIRECTORY + 'submission.csv')